# Module 08: Agentic RAG
This notebook demonstrates how traditional Chroma DB works for RAG pipelines.

## What we'll learn:
- ChromaDB
- OpenAI Embeddings
- RAG using State Machine
- Retrieval, Augment and Generation as steps

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [4]:
import os
import chromadb
from chromadb.utils import embedding_functions
from chromadb.api.models.Collection import Collection
import pdfplumber
from dotenv import load_dotenv
from typing import TypedDict, List

from lib.state_machine import StateMachine, Step, EntryPoint, Termination, Resource
from lib.llm import LLM
from lib.messages import BaseMessage, UserMessage, SystemMessage

In [5]:
import logging
logging.getLogger('pdfminer').setLevel(logging.ERROR)

In [6]:
load_dotenv()

True

In [7]:
sentence_list = [
    "Meta drops multimodal Llama 3.2 — here's why it's such a big deal",
    "Chip giant Nvidia acquires OctoAI, a Seattle startup that helps companies run AI models",
    "Google is bringing Gemini to all older Pixel Buds",
    "The first Intel Battlmage GPU benchmarks have leaked",
    "Dell partners with Nvidia to accelerate AI adoption in telecoms",
]
ids = ["id1", "id2", "id3", "id4", "id5"]

## ChromaDB with Default Embedding Function

In [8]:
chroma_client = chromadb.Client()

In [9]:
collection = chroma_client.create_collection(
    name="demo"
)

In [10]:
collection.add(
    documents=sentence_list,
    ids=ids
)

/Users/malathi.sankar/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 33.8MiB/s]


In [11]:
collection.count()

5

In [12]:
collection.peek(1)

{'ids': ['id1'],
 'embeddings': array([[ 6.06655143e-02, -3.51322480e-02,  6.06437288e-02,
         -5.11925854e-02,  1.13580197e-01, -1.88892763e-02,
         -2.68528331e-02,  5.48633784e-02,  3.23644504e-02,
          5.42442985e-02, -4.04198468e-02, -1.90558676e-02,
         -5.97919673e-02,  2.56032404e-02,  8.48459899e-02,
          4.12196927e-02,  3.95206176e-02, -4.00091521e-02,
         -7.66606703e-02,  2.78292280e-02,  5.38355187e-02,
         -1.35247353e-02,  9.65649709e-02, -3.04361507e-02,
          6.61455886e-03,  7.21730739e-02, -9.53866169e-02,
         -2.75959335e-02,  7.86793791e-03, -6.68519735e-02,
         -1.27341859e-02,  1.21338010e-01, -6.66138083e-02,
         -3.28670517e-02, -6.49284273e-02, -1.61901880e-02,
         -3.32960160e-03,  8.04080665e-02, -3.84503491e-02,
          1.37278825e-04,  3.72596853e-03,  4.83831167e-02,
         -3.65696087e-06, -4.51370180e-02, -1.37449345e-02,
         -7.15254620e-02,  1.01806317e-02, -4.23029736e-02,
         

In [13]:
collection.query(
    query_texts=["gadget"],
    n_results=2,
    include=['metadatas', 'documents', 'distances']
)

{'ids': [['id3', 'id1']],
 'embeddings': None,
 'documents': [['Google is bringing Gemini to all older Pixel Buds',
   "Meta drops multimodal Llama 3.2 — here's why it's such a big deal"]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[1.5251753330230713, 1.7548508644104004]]}

In [14]:
result = collection.query(
    query_texts=["gadget"],
    n_results=2,
    include=['metadatas', 'documents', 'distances']
)

result['documents'][0]

['Google is bringing Gemini to all older Pixel Buds',
 "Meta drops multimodal Llama 3.2 — here's why it's such a big deal"]

In [15]:
print(collection._embedding_function.name())

default


In [16]:
size = len(collection.peek(1)['embeddings'][0])
print(f"Size of the embeddings array: {size}")


Size of the embeddings array: 384


## OpenAI Embeddings

In [17]:
chroma_client.delete_collection(name="demo")

In [18]:
embeddings_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY")
)

In [19]:
collection = chroma_client.create_collection(
    name="demo",
    embedding_function=embeddings_fn
)

In [20]:
collection.add(
    documents=sentence_list,
    ids=ids
)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: voc-1454**************************************6929. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}} in add.

In [19]:
collection.query(
    query_texts=["gadget"],
    n_results=2,
    include=['metadatas', 'documents', 'distances']
)

{'ids': [['id3', 'id4']],
 'embeddings': None,
 'documents': [['Google is bringing Gemini to all older Pixel Buds',
   'The first Intel Battlmage GPU benchmarks have leaked']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[0.46601054072380066, 0.48678600788116455]]}

In [20]:
print(collection._embedding_function.name())

openai


In [21]:
size = len(collection.peek(1)['embeddings'][0])
print(f"Size of the embeddings array: {size}")

Size of the embeddings array: 1536


## RAG

**Load**

In [22]:
file_path = "GlobalEVOutlook2025.pdf"
documents = []
page_nums = []

In [23]:
with pdfplumber.open(file_path) as pdf:
    for num, page in enumerate(pdf.pages, start=1):
        text = page.extract_text()
        if text:
            documents.append(text)
            page_nums.append(str(num))


In [24]:
collection = chroma_client.create_collection(
    name="traditional_rag",
    embedding_function=embeddings_fn
)

In [25]:
collection.add(
    documents=documents,
    ids=page_nums
)

**State Machine**

In [26]:
class State(TypedDict):
    messages: List[BaseMessage]
    question: str
    documents: List[str]
    answer: str

**RAG: Retrieve**

In [27]:
def retrieve(state:State, resource:Resource):
    question = state["question"]
    collection:Collection = resource.vars.get("collection")
    results = collection.query(
        query_texts=[question],
        n_results=3,
        include=['documents']
    )
    retrieved_docs = results['documents'][0]
    
    return {"documents": retrieved_docs}

**RAG: Augment**

In [28]:
def augment(state:State):
    question = state["question"]
    documents = state["documents"]
    context = "\n\n".join(documents)

    messages = [
        SystemMessage(content="You are an assistant for question-answering tasks."),
        UserMessage(
            content=(
                "Use the following pieces of retrieved context to answer the question. "
                "If you don't know the answer, just say that you don't know. "
                f"\n# Question: \n-> {question} "
                f"\n# Context: \n-> {context} "
                "\n# Answer: "
            )
        )
    ]

    return {"messages": messages}

**RAG: Generate**

In [29]:
def generate(state:State, resource:Resource):
    llm:LLM = resource.vars.get("llm")
    ai_message = llm.invoke(state["messages"])
    return {
        "answer": ai_message.content, 
        "messages": state["messages"] + [ai_message],
    }

In [30]:
workflow = StateMachine(State)

In [31]:
# Create steps
entry = EntryPoint()
retrieve_step = Step("retrieve", retrieve)
augment_step = Step("augment", augment)
generate_step = Step("generate", generate)
termination = Termination()
        
workflow.add_steps(
    [
        entry, 
        retrieve_step, 
        augment_step, 
        generate_step, 
        termination
    ]
)

In [32]:
# Add transitions
workflow.connect(entry, retrieve_step)
workflow.connect(retrieve_step, augment_step)
workflow.connect(augment_step, generate_step)
workflow.connect(generate_step, termination)

In [33]:
llm = LLM(
    model="gpt-4o-mini",
    temperature=0.3,
)

In [34]:
resource = Resource(
    vars = {
        "llm": llm,
        "collection": collection,
    }
)

In [35]:
initial_state: State = {
    "question": "What was the number of electric car sales and their market share in Brazil in 2024?",
}

In [36]:
run_object = workflow.run(initial_state, resource)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__


In [37]:
run_object.get_final_state()["answer"]

'In 2024, Brazil had nearly 125,000 electric car sales, which represented a market share of 6.5%.'